# Data Cleaning and Preparation

In this notebook, we will perform data cleaning and preparation for all resorts across the Alps. This includes:

- Loading the raw data
- Handling missing values
- Correcting data types
- Normalizing resort names to handle special characters
- Filtering data based on resort operating dates
- Saving the cleaned data for further analysis

## 1. Import Libraries

In [1]:
import pandas as pd
import os
import unicodedata
import re
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

## 2. Handling Special Characters in File and Resort Names

To avoid issues with special characters (like accents and apostrophes) in file names and resort names, we'll define a normalization function. This function will:

- Convert names to lowercase
- Remove accents and diacritics
- Replace non-alphanumeric characters with underscores

In [2]:
def normalize_name(name):
    """
    Normalize names by converting to lowercase, removing accents, and replacing non-alphanumeric characters with underscores.
    """
    # Convert to lowercase
    name = name.lower()
    # Remove accents and diacritics
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    # Replace non-alphanumeric characters with underscores
    name = re.sub(r'[^a-z0-9]+', '_', name)
    # Remove leading/trailing underscores
    name = name.strip('_')
    return name

### 2 (a) Utility function for standardising columns.

In [3]:
def standardize_columns(df, dataset_type):
    """
    Standardize column names based on dataset type.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to standardize.
    - dataset_type (str): 'old' or 'new'.
    
    Returns:
    - pd.DataFrame: Standardized DataFrame.
    """
    if dataset_type == 'new':
        # Rename 'time' to 'date'
        df = df.rename(columns={'time': 'date'})
        # No renaming needed for other columns
    elif dataset_type == 'old':
        # Rename columns to standard names
        column_mappings = {
            'temperature_2m_max': 'temperature_max',
            'temperature_2m_min': 'temperature_min',
            'rain_sum': 'precipitation_sum',
        }
        df = df.rename(columns=column_mappings)
    return df

### 2 (b) Set data paths

In [4]:
raw_data_root = '../data/raw/cds'
processed_data_root = '../data/processed/cds'

### 3. Function to get list of all CSV files in the raw data directory

We'll create a function to traverse the directory structure and collect all CSV files. While doing so, we'll normalize the country and resort names to ensure consistency.

In [5]:
def get_all_csv_files_with_metadata(root_dir):
    csv_files = []
    for country in os.listdir(root_dir):
        country_path = os.path.join(root_dir, country)
        if os.path.isdir(country_path):
            normalized_country = normalize_name(country)
            for resort in os.listdir(country_path):
                resort_path = os.path.join(country_path, resort)
                if os.path.isdir(resort_path):
                    normalized_resort = normalize_name(resort)
                    for file in os.listdir(resort_path):
                        if file.endswith('.csv'):
                            file_path = os.path.join(resort_path, file)
                            dataset_type = 'unknown'
                            try:
                                df_sample = pd.read_csv(file_path, nrows=1)
                                if 'time' in df_sample.columns:
                                    dataset_type = 'new'
                                elif 'date' in df_sample.columns:
                                    dataset_type = 'old'
                                else:
                                    print(f"File {file_path} does not contain 'time' or 'date' column. Skipping.")
                                    continue  # Skip this file
                            except Exception as e:
                                print(f"Error reading {file_path}: {e}")
                                continue  # Skip this file
                            
                            csv_files.append({
                                'type': dataset_type,  # 'old' or 'new'
                                'country': normalized_country,
                                'resort': normalized_resort,
                                'file_path': file_path
                            })
    return csv_files


## 4. Data Cleaning Steps

We will perform the following data cleaning steps for each resort:

1. Remove empty rows prior to `2021-03-23`
2. Handle missing values
3. Handle duplicates
4. Correct data types
5. Filter data based on each resort's opening and closing dates
6. Save cleaned data

## 4.1 Function to Clean and Filter a Single CSV File

In [7]:
def clean_and_filter_data(file_info):
    """
    Cleans and filters data based on dataset type.
    
    Parameters:
    - file_info (dict): Information about the file.
    
    Returns:
    - key (str): Unique key for the resort.
    - df (pd.DataFrame): Cleaned DataFrame.
    """

    country = file_info['country']
    resort = file_info['resort']
    dataset_type = file_info['type']  # 'old' or 'new'
    file_path = file_info['file_path']
    
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Standardize column names based on dataset type
        df = standardize_columns(df, dataset_type)
        
        # Convert 'date' column to datetime format and remove timezone info
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            if dataset_type == 'old':
                # Remove timezone information if present
                df['date'] = df['date'].dt.tz_localize(None) if df['date'].dt.tz else df['date']
        else:
            print(f"'date' column not found in {file_path}. Skipping date parsing.")
            df['date'] = pd.NaT  # Assign NaT if 'date' is missing
    
        df = df.dropna(subset=['date'])
        if dataset_type == 'old':
            # Remove rows prior to 2021-03-23
            df = df[df['date'] >= '2021-03-23']
    
        df = df.reset_index(drop=True)
        key = f"{country}/{resort}"
    
        return key, df
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None, None

### 4.2 Process All CSV Files

In [8]:
# Define the root directory
raw_data_root = '../data/raw/cds'

# Get list of all CSV files with dataset type
csv_files = get_all_csv_files_with_metadata(raw_data_root)
print(f"Found {len(csv_files)} CSV files (Old: {sum(1 for f in csv_files if f['type']=='old')}, New: {sum(1 for f in csv_files if f['type']=='new')}).")

data_frames = {}

for file_info in csv_files:
    key, df = clean_and_filter_data(file_info)
    if key and df is not None:
        data_frames[key] = df
        print(f"Loaded and cleaned data for {key}: {df.shape[0]} rows.")
    else:
        print(f"Failed to process {file_info['file_path']}.")

Found 30 CSV files (Old: 24, New: 6).
Loaded and cleaned data for austrian_alps/kitzbuhel: 11184 rows.
Loaded and cleaned data for austrian_alps/kitzbuhel: 1166 rows.
Loaded and cleaned data for austrian_alps/st_anton: 12418 rows.
Loaded and cleaned data for austrian_alps/st_anton: 1166 rows.
Loaded and cleaned data for austrian_alps/solden: 12418 rows.
Loaded and cleaned data for austrian_alps/solden: 1166 rows.
Loaded and cleaned data for french_alps/chamonix: 1166 rows.
Loaded and cleaned data for french_alps/chamonix: 12418 rows.
Loaded and cleaned data for french_alps/les_trois_vallees: 12418 rows.
Loaded and cleaned data for french_alps/les_trois_vallees: 1166 rows.
Loaded and cleaned data for french_alps/val_d_isere_tignes: 12418 rows.
Loaded and cleaned data for french_alps/val_d_isere_tignes: 1166 rows.
Loaded and cleaned data for italian_alps/cortina_d_ampezzo: 1166 rows.
Loaded and cleaned data for italian_alps/cortina_d_ampezzo: 1014 rows.
Loaded and cleaned data for italia

## 4.3 Filter data based on each resort's opening and closing dates

Each resort operates during specific dates in the year. We'll filter the data to include only the dates when each resort is open.

Here are the approximate opening and closing dates for each resort:

- **French Alps:**
  - **Chamonix:** Opens mid-December (`12-15`), closes mid-May (`05-15`)
  - **Val d'Isère & Tignes:** Opens November 30 (`11-30`), closes May 5 (`05-05`)
  - **Les Trois Vallées:** Opens December 7 (`12-07`), closes mid-April (`04-15`)
  
- **Austrian Alps:**
  - **St. Anton:** Opens early December (`12-01`), closes late April (`04-30`)
  - **Kitzbühel:** Opens mid-October (`10-15`), closes May (`05-01`)
  - **Sölden:** Opens early November (`11-01`), closes early May (`05-01`)
  
- **Swiss Alps:**
  - **Zermatt:** Opens mid-November (`11-15`), closes late April (`04-30`)
  - **St. Moritz:** Opens late November (`11-25`), closes early May (`05-01`)
  - **Verbier:** Opens early December (`12-01`), closes late April (`04-30`)
  
- **Italian Alps:**
  - **Cortina d'Ampezzo:** Opens late November (`11-25`), closes early April (`04-05`)
  - **Val Gardena:** Opens early December (`12-01`), closes mid-April (`04-15`)
  - **Sestriere:** Opens early December (`12-01`), closes mid-April (`04-15`)
  
- **Slovenian Alps:**
  - **Kranjska Gora:** Opens mid-December (`12-15`), closes mid-April (`04-15`)
  - **Mariborsko Pohorje:** Opens December (`12-01`), closes early April (`04-05`)
  - **Krvavec:** Opens December (`12-01`), closes April (`04-30`)

  We'll define the `resort_seasons` dictionary with normalized keys to match the keys in `data_frames`.


In [9]:
resort_seasons = {
    'french_alps/chamonix': {'open': '12-15', 'close': '05-15'},
    'french_alps/val_d_isere_tignes': {'open': '11-30', 'close': '05-05'},
    'french_alps/les_trois_vallees': {'open': '12-07', 'close': '04-15'},
    'austrian_alps/st_anton': {'open': '12-01', 'close': '04-30'},
    'austrian_alps/kitzbuhel': {'open': '10-15', 'close': '05-01'},
    'austrian_alps/solden': {'open': '11-01', 'close': '05-01'},
    'swiss_alps/zermatt': {'open': '11-15', 'close': '04-30'},
    'swiss_alps/st_moritz': {'open': '11-25', 'close': '05-01'},
    'swiss_alps/verbier': {'open': '12-01', 'close': '04-30'},
    'italian_alps/cortina_d_ampezzo': {'open': '11-25', 'close': '04-05'},
    'italian_alps/val_gardena': {'open': '12-01', 'close': '04-15'},
    'italian_alps/sestriere': {'open': '12-01', 'close': '04-15'},
    'slovenian_alps/kranjska_gora': {'open': '12-15', 'close': '04-15'},
    'slovenian_alps/mariborsko_pohorje': {'open': '12-01', 'close': '04-05'},
    'slovenian_alps/krvavec': {'open': '12-01', 'close': '04-30'},
}


### 4.4 Handles seasons that span across years

In [10]:
def get_season_dates(year, open_mm_dd, close_mm_dd):
    """
    Given a year and open/close month-day strings, return datetime objects for open and close dates.
    Handles seasons that span across years.
    """
    open_month, open_day = map(int, open_mm_dd.split('-'))
    close_month, close_day = map(int, close_mm_dd.split('-'))
    
    open_date = pd.Timestamp(year=year, month=open_month, day=open_day)
    close_date = pd.Timestamp(year=year, month=close_month, day=close_day)
    
    # If close_date is earlier than open_date, it spans to the next year
    if close_date < open_date:
        close_date += pd.DateOffset(years=1)
    
    return open_date, close_date

### 4.5 Assign rows to dataframe to a season based on the operating dates.

In [ ]:
def categorize_season(df, season_info, resort_key):
    """
    Parameters:
    - df (pd.DataFrame): DataFrame containing 'date' column.
    - season_info (dict): Dictionary with 'open' and 'close' dates in 'MM-DD' format.
    - resort_key (str): Key to identify the resort (e.g., 'french_alps/chamonix').
    
    Returns:
    - pd.DataFrame: DataFrame with an added 'season_id' column.
    """
    if not season_info:
        # No season information provided
        df['season_id'] = None
        return df
    
    open_mm_dd = season_info['open']
    close_mm_dd = season_info['close']
    
    df = df.copy()
    df['season_id'] = None  # Initialize season identifier
    
    years = df['date'].dt.year.unique()
    
    for year in years:
        open_date, close_date = get_season_dates(year, open_mm_dd, close_mm_dd)
        
        # Filter rows within the current season
        season_mask = (df['date'] >= open_date) & (df['date'] <= close_date)
        season_label = f"{year}-{close_date.year}"
        
        df.loc[season_mask, 'season_id'] = season_label
    
    return df

### 4.7 Apply Season Categorization

In [ ]:
for key, df in data_frames.items():
    resort = key
    if resort in resort_seasons:
        season_info = resort_seasons[resort]
        
        # Categorize seasons
        df = categorize_season(df, season_info, resort)
        
        # Update the DataFrame in the dictionary
        data_frames[key] = df
        print(f"Season categorized for {resort}.")
    else:
        print(f"No season information for {resort}. Data not categorized.")

### 4.8 Add Operating Season Indicator

In [ ]:
def add_operating_season_indicator(df):
    """
    Adds a boolean column 'is_operating_season' indicating if the row is within an operating season.
    """
    df = df.copy()
    df['is_operating_season'] = df['season_id'].notnull()
    return df

for key, df in data_frames.items():
    df = add_operating_season_indicator(df)
    data_frames[key] = df
    print(f"Operating season indicator added for {key}.")

### 4.9 Handle snow_depth and snowfall_sum Separately

Since snow_depth and snowfall_sum are distinct metrics, handle them based on dataset type.

In [ ]:
for key, df in data_frames.items():
    dataset_type = 'new' if 'snow_depth' in df.columns else 'old'
    
    if dataset_type == 'new':
        # Handle 'snow_depth'
        # Set 'snow_depth' to zero outside operating seasons
        df.loc[~df['is_operating_season'], 'snow_depth'] = 0
        
        # Impute missing 'snow_depth' within operating seasons
        # Example: Mean Imputation
        mean_snow_depth = df[df['is_operating_season']]['snow_depth'].mean()
        df.loc[df['is_operating_season'] & df['snow_depth'].isnull(), 'snow_depth_imputed'] = mean_snow_depth
        
    elif dataset_type == 'old':
        # Handle 'snowfall_sum'
        # Set 'snowfall_sum' to zero outside operating seasons
        df.loc[~df['is_operating_season'], 'snowfall_sum'] = 0
        
        # Impute missing 'snowfall_sum' within operating seasons
        # Example: Mean Imputation
        mean_snowfall_sum = df[df['is_operating_season']]['snowfall_sum'].mean()
        df.loc[df['is_operating_season'] & df['snowfall_sum'].isnull(), 'snowfall_sum_imputed'] = mean_snowfall_sum
    
    data_frames[key] = df
    print(f"Handled snow metrics for {key}.")


### 4.10  Impute Missing Values Appropriately

Implement imputation methods based on dataset type.

## 4.4/5 Filter Data Based on Resort Operating Dates

We'll filter each resort's data to include only dates within its operating season.

In [ ]:
for key, df in data_frames.items():
    resort = key
    if resort in resort_seasons:
        season = resort_seasons[resort]
        open_month_day = season['open']
        close_month_day = season['close']
        
        # Since the data spans multiple years, we need to filter for each year
        df['year'] = df['date'].dt.year
        filtered_dfs = []
        
        for year in df['year'].unique():
            open_date_str = f"{year}-{open_month_day}"
            close_date_str = f"{year}-{close_month_day}"
            open_date = pd.to_datetime(open_date_str, errors='coerce').tz_localize(None)
            close_date = pd.to_datetime(close_date_str, errors='coerce').tz_localize(None)
            
            # Handle cases where the season spans over the new year
            if close_date < open_date:
                # Season spans over to the next year
                close_date += relativedelta(years=1)
            
            season_df = df[(df['date'] >= open_date) & (df['date'] <= close_date)]
            filtered_dfs.append(season_df)
        
        # Combine all seasons
        df_season = pd.concat(filtered_dfs)
        
        # Drop the 'year' column
        df_season = df_season.drop(columns=['year'])
        
        # Update the DataFrame in the dictionary
        data_frames[key] = df_season.reset_index(drop=True)
        
        print(f"Filtered data for {resort}: {df_season.shape[0]} rows within operating dates.")
    else:
        print(f"No season information for {resort}. Data not filtered.")

## 5. Save Cleaned Data

We'll save the cleaned and filtered DataFrames to the `data/processed` directory, maintaining the normalized folder structure.

In [ ]:
for key, df in data_frames.items():
    # Split the key back into country and resort
    country, resort = key.split('/')
    # Build the processed data path
    processed_dir = os.path.join(processed_data_root, country, resort)
    os.makedirs(processed_dir, exist_ok=True)
    # Save the cleaned DataFrame
    processed_file_path = os.path.join(processed_dir, f"{resort}_cleaned.csv")
    df.to_csv(processed_file_path, index=False)
    print(f"Saved cleaned data to {processed_file_path}.")

## 6. Summary

- Loaded and cleaned data for all resorts.
- Normalized resort names to handle special characters.
- Filtered data based on each resort's operating dates.
- Saved cleaned data to the `data/processed` directory.

These datasets are now ready for feature engineering and further analysis.

We have further data to analyseand clean.  Furthermore, data impudation may be required following aforemtnioend analysis.